<p align="center" ><img src="https://www.ai4kids.ai/wp-content/uploads/2019/07/ai4kids_website_logo_120x40.png"></img></p>

# AI 專題實作：貓狗辨識

貓狗辨識專題使用的是 **限時塗鴉 (Quick, Draw!)** 提供的貓與狗資料集。官方提供 4 種不同的資料格式，在本專題實作中將使用的是 NumPy 陣列 (npy)。

- 資料集說明可以參考官方 GitHub：[The Quick, Draw! Dataset](https://github.com/googlecreativelab/quickdraw-dataset)

- 限時塗鴉體驗網站：[https://quickdraw.withgoogle.com/](https://quickdraw.withgoogle.com/)

<p align="right">© Copyright AI4kids.ai</p>

## 1. 環境準備

執行下列 Cell 載入專題需要使用的套件。
* **tensorflow**：Google 團隊開發的機器學習框架套件，可用以開發各種感知和語言理解任務的機器學習。
* **numpy**：一個 Python 的擴充套件，支援高階維度的陣列與矩陣運算。
* **pandas**：一個資料分析套件，提供高效、簡易的資料格式(Data Frame)讓開發者可以快速操作與分析資料。 
* **matplotlib.pyplot**：繪圖套件包。
* **train_test_split**：是 Scikit-Learn 套件中的一個函式，可用以對資料集進行切分，例如 67% 作為訓練使用，33% 作為測試使用。


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


## 2. 下載資料集

Google 將開放的限時塗鴉資料集存放在 Google Cloud Platform 的 Storage，從網址 [https://console.cloud.google.com/storage/browser/quickdraw_dataset/full](https://console.cloud.google.com/storage/browser/quickdraw_dataset/full)   
可以瀏覽所有資料集。  

在專題中，執行下列指令直接下載，並儲存在 Colab 中。  

<img src="https://i.imgur.com/2Vjth88.png" width="300">


In [ ]:
import urllib.request

urllib.request.urlretrieve('https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/cat.npy', 'cat.npy')
urllib.request.urlretrieve('https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/dog.npy', 'dog.npy')

瀏覽已下載檔案

In [ ]:
!ls -l

## 3. 載入資料集與資料預處理

將貓、狗的資料集載入至 NumPy 陣列。

In [ ]:
#dog = np.load('dog.npy',encoding='bytes', allow_pickle=True)
#cat = np.load('cat.npy',encoding='bytes', allow_pickle=True)

dog = np.load('dog.npy')
cat = np.load('cat.npy')


載入後資料型別為 NumPy 陣列，查看 `shape` 屬性可以看到 dog 與 cat 分別有 152159 與 123202 筆資料 (也就是有 152159 與 123202 張圖片)。

原始圖片的大小為 28x28，在資料集中將其攤平為一維的維度，所以每筆共有 784 個資料點。

In [ ]:
dog

In [ ]:
dog.shape

In [ ]:
dog[0]

In [ ]:
cat.shape

使用 Matplotlib 來將陣列資料顯示為原始圖檔，在顯示之前先將資料點維度回復成為二維 28x28 的陣列。

可以看到原始圖檔如下。

In [ ]:
fig=plt.figure(figsize=(10, 10))
columns = 5
rows = 1

for i in range(1, columns * rows + 1):
    
    img = dog[i].reshape(28,28)
    
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
#    plt.imshow(img, cmap='gray')

plt.show()

PS：上圖資料集應該為灰階點陣圖，輸出成類似彩色的結果是因為`imshow()`指令預設為彩色，因此程式會嘗試將灰階圖映射至彩色，就成為上面顯示的`假色圖(Pseudocolor Image)`，
可試試使用`plt.imshow(img, cmap='gray')`。

在專題中，我們僅取狗與貓資料的前 60000 筆來進行訓練及測試。

In [ ]:
sample_size = 60000

將狗與貓資料合併，成為同一個輸入資料。   
<img src="https://i.imgur.com/21NVDcV.png" width="600">

In [ ]:
X = np.concatenate((dog[:sample_size], cat[:sample_size]))

將資料集資料重塑為原始圖檔的維度，並且將各個元素值除以 255.0 進行**正規化 (Normalization)**處理。   
reshape() 方法中傳入 4 個參數，後半部的 (28, 28, 1) 表示轉換後的形狀要為 `28x28x1` 的陣列，最後一個參數代表的是圖片`通道數 (Channel)`，因圖片資料為灰階圖，因此設定為 1。   
第一個參數-1，表示讓python**自動計算**此樣本維度。

In [ ]:
X = X.reshape((-1,28,28,1))/255.0

In [ ]:
#X = np.concatenate((dog[:sample_size], cat[:sample_size]))

In [ ]:
#X = X / 255.0
#X = X.reshape(-1,28,28,1)

In [ ]:
X.shape

In [ ]:
Y = np.zeros(2 * sample_size)

In [ ]:
Y[sample_size:] = 1.0

建立資料的類別標籤，狗的類別標籤為 0，貓的類別標籤為 1。
<img src="https://i.imgur.com/250kw8W.png" width="600">

In [ ]:
Y.shape

在開始建立模型前，使用 Scikit-Learn 的 `train_test_split` 函式將 120000 筆資料順序隨機處理後，切分為訓練集和測試集。這邊設定的比例是訓練集 70%、測試集 30% 的切分比例。

切分後回傳得到訓練集資料、訓練集類別標籤、測試集資料、測試集類別標籤。   

* **X**和**Y**就是上面步驟準備好的原始資料，分別表示 dog、cat 合併後的輸入資料集，與各資料項對應的答案標籤 (Label)。 
* **random_state**就是亂數種子，相同的亂數種子在不同 Cell 執行會有相同結果。若設定為零，就表示為隨機切分。 
* **test_size**若設定為浮點數，表示測試資料要佔全部資料的多少百分比 ; 若設定為整數，表示為多少筆測試資料。 

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(X, Y, random_state=41, test_size=0.3)

<img src="https://i.imgur.com/WgOJpAr.png" width="800">

<img src="https://i.imgur.com/ADAPcaQ.png" width="600">

In [ ]:
print(train_x.shape)
print(test_x.shape)

類別標籤轉換為 one hot 編碼 (encoding)

<img src="https://i.imgur.com/twjaBXn.png" width="600">

In [ ]:
print(f"{train_y[0]} \n{test_y[0]}")

In [ ]:
train_y = tf.keras.utils.to_categorical(train_y)
test_y = tf.keras.utils.to_categorical(test_y)

In [ ]:
print(f"{train_y[0]} \n{test_y[0]}")

## 4. 創建 CNN 類神經網路並訓練模型
執行完資料集的前處理後，接著就可以利用**Keras**開始創建網路模型。Keras 是一個用 Python 開發的高階深度學習函式庫，透過此函式庫可以實作下面 步 驟：**定義 (Define)**、 **編譯 (Compile)**、 **訓練 (Fit)**、 **評估 (Evaluate)** 及**預測 (Prediction)** 

在訓練模型前，須先定義並編譯 (compile) 模型。

下面定義了有 2 個卷積層 (`Conv2D()`)、2 個池化層 (`MaxPooling2D()`)、1 個平坦層 (`Flatten()`)、2 個全連接層 (`Dense()`) 的 CNN 網路。   

**Convolution**與**Pooling**的運算機制請參考投影片內容。
 

<img src="https://i.imgur.com/9u0OHZJ.png">

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), input_shape=(28,28,1), activation='relu'), 
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(), 
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(2,activation='softmax')
])

下面指令，可輸出目前模型結構圖：

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, rankdir='LR')

下面指令，可看到目前所創建的網路結構的摘要資訊：

In [ ]:
model.summary()

**Softmax 說明**

> Softmax函數，是邏輯函數的一種擴充。它能將一個含任意實數的K維向量，「壓縮」到另一個K維實向量中，使得每一個元素的範圍都在(0,1)之間，並且所有元素的和為 1

Softmax 公式

> <img src="https://i.imgur.com/HoHPLCF.png" width="200">

Softmax 例子

> 輸入向量
> 
> - [1, 2, 3, 4, 1, 2, 3]
> 
> 對應的Softmax函數的值為
>
> - [0.024, 0.064, 0.175, 0.475, 0.024, 0.064, 0.175]
>
> 輸出向量中擁有最大權重的項對應著輸入向量中的最大值「4」。這也顯示了這個函數通常的意義：對向量進行標準化（歸一化），凸顯其中最大的值並抑制遠低於最大值的其他分量。

模型編譯部分，使用 Categorical Crossentropy 做為損失函數，優化器為 Adam，並以預測準確率 (accuracy) 做為模型表現在衡量指標。  

可視化優化器 - SGD / RMSProp  (Adam 是  RMSProp 的加強版) 資料來源  https://ruder.io/optimizing-gradient-descent/

<img src="https://ruder.io/content/images/2016/09/contours_evaluation_optimizers.gif" width="450">

<img src="https://ruder.io/content/images/2016/09/saddle_point_evaluation_optimizers.gif" width="450">

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['acc'])

使用訓練集資料與訓練集類別標籤進行模型訓練，在訓練過程中並設定使用 20% 的資料做為驗證 (validation) 資料，進行交叉驗證。

* x：設定學習資料，範例中設定訓練用的圖片資料陣列 train_x。    
* y：設定答案資料，範例中設定訓練用的圖片答案資料陣列 train_y。   
* batch_size：設定批次大小，範例中設定為 128。 
* epochs：設定 epoch(= 學習次數)，範例中設定為 20。 
* verbose：設定是否顯示日誌(0、1)，預設為 1。設定為 0 表示不在標準輸出顯示日誌。 
* validation_split：設定驗證資料集比例，此比例表示要從訓練資料集中取百分之幾做為驗證資料集，用以測試所選參數用於該模型的效果，範例中設定為 0.2。 

<img src="https://i.imgur.com/S0j28qF.png" width="600">

In [ ]:
history = model.fit(x=train_x, y=train_y, 
                    batch_size=128, epochs=20, 
                    verbose=1, validation_split=0.2)

關於 **Overfitting**的說明

<img src="https://i.imgur.com/p0lIi0G.png" width="600">
<img src="https://i.imgur.com/3ryFn3Z.png" width="600">



In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

## 5.測試與驗證準確率

訓練結束後，使用測試資料集與測試集類別標籤評估訓練後的模型。

In [ ]:
model.evaluate(test_x, test_y)

In [ ]:
手動選擇一個圖片，查看預測結果是否正確。

執行後在輸入框輸入任一個圖片編號 ($\leq$ 35999，小於測試集的筆數中任一個數字)。

In [ ]:
test_no = int(input('please input photo number of test data: '))
plt.imshow(test_x[test_no].reshape(28,28), cmap="gray")

print(f'it is {np.argmax(test_y[test_no])}')
pred = model.predict_classes(test_x[test_no:test_no+1])
print(f'predict it is {pred}')

In [ ]:
# dog_cat={0:'a dog', 1:'a cat'}
dog_cat2 =['a dog', 'a cat']

test_no = int(input('please input photo number of test data: '))
plt.imshow(test_x[test_no].reshape(28,28), cmap="gray")

print(f'it is {dog_cat2[np.argmax(test_y[test_no])]}')
pred = model.predict_classes(test_x[test_no:test_no+1])
print(f'predict it is {dog_cat2[int(pred)]}')

## 6. 修正過擬合 (Overfitting) 問題

雖然在上面的訓練過程過後似乎得到不錯的預測準確率 (約98%)，但是從結果卻可以看出來我們的模型在針對訓練過程中沒有看過的資料 (例如驗證資料集validate set與測試資料集testing set) 時準確率就會有所下降 (約89%)，這個現象就是過擬合 (Overfitting) 問題。

在下面的範例中，我們採用其中一種解決過擬合問題的技巧，在模型中加入正規化 (Regularization)，避免模型在訓練的過程中**死背**答案造成過擬合。

In [ ]:
model2 = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), input_shape=(28,28,1), activation='relu'), 
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Flatten(), 
  tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(2, activation='softmax')
])

In [ ]:
model2.summary()

In [ ]:
model2.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['acc'])
history = model2.fit(x=train_x, y=train_y, batch_size=128, epochs=50, verbose=1, validation_split=0.2)

從訓練及驗證的結果我們可以看到，隨著訓練的時間拉長，訓練及驗證的結果都有逐步改善，也沒有過擬合的現象，代表新的模型泛化 (generalization) 能力變強，遇到新的資料時預測能力會較好。

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
model.evaluate(test_x, test_y)